# 9 WorkFlow Analista Senior

### 9.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 9.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

## 9.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Jul 14 12:36:44 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656917,35.1,1439322,76.9,1431277,76.5
Vcells,1225041,9.4,8388608,64.0,1924957,14.7


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191

PARAM$experimento <- "SEMILLERIO_04"
PARAM$dataset <- "analistasr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 9.3.1   Preprocesamiento del dataset

#### 9.3.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 9.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [7]:
if( !require("mice")) install.packages("mice", repos = "http://cran.us.r-project.org")
require("mice")

Loading required package: mice


Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind




In [8]:
# Escrito por alumnos de  Universidad Austral  Rosario

Corregir_MICE <- function(pcampo, pmeses) {

  meth <- rep("", ncol(dataset))
  names(meth) <- colnames(dataset)
  meth[names(meth) == pcampo] <- "sample"

  # llamada a mice  !
  imputacion <- mice(dataset,
    method = meth,
    maxit = 5,
    m = 1,
    seed = 7)

  tbl <- mice::complete(dataset)

  dataset[, paste0(pcampo) := ifelse(foto_mes %in% pmeses, tbl[, get(pcampo)], get(pcampo))]

}


In [9]:
Corregir_interpolar <- function(pcampo, pmeses) {

  tbl <- dataset[, list(
    "v1" = shift(get(pcampo), 1, type = "lag"),
    "v2" = shift(get(pcampo), 1, type = "lead")
  ),
  by = eval(envg$PARAM$dataset_metadata$entity_id)
  ]

  tbl[, paste0(envg$PARAM$dataset_metadata$entity_id) := NULL]
  tbl[, promedio := rowMeans(tbl, na.rm = TRUE)]

  dataset[
    ,
    paste0(pcampo) := ifelse(!(foto_mes %in% pmeses),
      get(pcampo),
      tbl$promedio
    )
  ]
}

In [10]:
AsignarNA_campomeses <- function(pcampo, pmeses) {

  if( pcampo %in% colnames( dataset ) ) {

    dataset[ foto_mes %in% pmeses, paste0(pcampo) := NA ]
  }
}

In [11]:

Corregir_atributo <- function(pcampo, pmeses, pmetodo)
{
  # si el campo no existe en el dataset, Afuera !
  if( !(pcampo %in% colnames( dataset )) )
    return( 1 )

  # llamo a la funcion especializada que corresponde
  switch( pmetodo,
    "MachineLearning"     = AsignarNA_campomeses(pcampo, pmeses),
    "EstadisticaClasica"  = Corregir_interpolar(pcampo, pmeses),
    "MICE"                = Corregir_MICE(pcampo, pmeses),
  )

  return( 0 )
}

In [12]:

Corregir_Rotas <- function(dataset, pmetodo) {
  gc(verbose= FALSE)
  cat( "inicio Corregir_Rotas()\n")
  # acomodo los errores del dataset

  Corregir_atributo("active_quarter", c(202006), pmetodo) # 1
  Corregir_atributo("internet", c(202006), pmetodo) # 2

  Corregir_atributo("mrentabilidad", c(201905, 201910, 202006), pmetodo) # 3
  Corregir_atributo("mrentabilidad_annual", c(201905, 201910, 202006), pmetodo) # 4

  Corregir_atributo("mcomisiones", c(201905, 201910, 202006), pmetodo) # 5

  Corregir_atributo("mactivos_margen", c(201905, 201910, 202006), pmetodo) # 6
  Corregir_atributo("mpasivos_margen", c(201905, 201910, 202006), pmetodo) # 7

  Corregir_atributo("mcuentas_saldo", c(202006), pmetodo) # 8

  Corregir_atributo("ctarjeta_debito_transacciones", c(202006), pmetodo) # 9

  Corregir_atributo("mautoservicio", c(202006), pmetodo) # 10

  Corregir_atributo("ctarjeta_visa_transacciones", c(202006), pmetodo) # 11
  Corregir_atributo("mtarjeta_visa_consumo", c(202006), pmetodo) # 12

  Corregir_atributo("ctarjeta_master_transacciones", c(202006), pmetodo) # 13
  Corregir_atributo("mtarjeta_master_consumo", c(202006), pmetodo) # 14

  Corregir_atributo("ctarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 15
  Corregir_atributo("mttarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 16

  Corregir_atributo("ccajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 17

  Corregir_atributo("mcajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 18

  Corregir_atributo("ctarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 19

  Corregir_atributo("mtarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 20

  Corregir_atributo("ctarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 21

  Corregir_atributo("mtarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 22

  Corregir_atributo("ccomisiones_otras", c(201905, 201910, 202006), pmetodo) # 23
  Corregir_atributo("mcomisiones_otras", c(201905, 201910, 202006), pmetodo) # 24

  Corregir_atributo("cextraccion_autoservicio", c(202006), pmetodo) # 25
  Corregir_atributo("mextraccion_autoservicio", c(202006), pmetodo) # 26

  Corregir_atributo("ccheques_depositados", c(202006), pmetodo) # 27
  Corregir_atributo("mcheques_depositados", c(202006), pmetodo) # 28
  Corregir_atributo("ccheques_emitidos", c(202006), pmetodo) # 29
  Corregir_atributo("mcheques_emitidos", c(202006), pmetodo) # 30
  Corregir_atributo("ccheques_depositados_rechazados", c(202006), pmetodo) # 31
  Corregir_atributo("mcheques_depositados_rechazados", c(202006), pmetodo) # 32
  Corregir_atributo("ccheques_emitidos_rechazados", c(202006), pmetodo) # 33
  Corregir_atributo("mcheques_emitidos_rechazados", c(202006), pmetodo) # 34

  Corregir_atributo("tcallcenter", c(202006), pmetodo) # 35
  Corregir_atributo("ccallcenter_transacciones", c(202006), pmetodo) # 36

  Corregir_atributo("thomebanking", c(202006), pmetodo) # 37
  Corregir_atributo("chomebanking_transacciones", c(201910, 202006), pmetodo) # 38

  Corregir_atributo("ccajas_transacciones", c(202006), pmetodo) # 39
  Corregir_atributo("ccajas_consultas", c(202006), pmetodo) # 40

  Corregir_atributo("ccajas_depositos", c(202006, 202105), pmetodo) # 41

  Corregir_atributo("ccajas_extracciones", c(202006), pmetodo) # 41
  Corregir_atributo("ccajas_otras", c(202006), pmetodo) # 43

  Corregir_atributo("catm_trx", c(202006), pmetodo) # 44
  Corregir_atributo("matm", c(202006), pmetodo) # 45
  Corregir_atributo("catm_trx_other", c(202006), pmetodo) # 46
  Corregir_atributo("matm_other", c(202006), pmetodo) # 47

  cat( "fin Corregir_rotas()\n")
}


In [13]:
# resuelvo el Catastrophe Analysis

setorder( dataset, numero_de_cliente, foto_mes )

PARAM$CA$metodo= "MachineLearning"

if( PARAM$CA$metodo %in% c("MachineLearning", "EstadisticaClasica", "MICE") )
  Corregir_Rotas(dataset, PARAM$CA$metodo)

inicio Corregir_Rotas()
fin Corregir_rotas()


#### 9.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [14]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)


In [15]:
# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.9903030878, 1.9174403544, 1.8296186587,
  1.7728862972, 1.7212488323, 1.6776304408,
  1.6431248196, 1.5814483345, 1.4947526791,
  1.4484037589, 1.3913580777, 1.3404220402,
  1.3154288912, 1.2921698342, 1.2472681797,
  1.2300475145, 1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219, 0.7566381305, 0.7289384687
)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.380952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vUVA <- c(
  2.001408838932958,  1.950325472789153,  1.89323032351521,
  1.8247220405493787, 1.746027787673673,  1.6871348409529485,
  1.6361678865622313, 1.5927529755859773, 1.5549162794128493,
  1.4949100586391746, 1.4197729500774545, 1.3678188186372326,
  1.3136508617223726, 1.2690535173062818, 1.2381595983200178,
  1.211656735577568,  1.1770808941405335, 1.1570338657445522,
  1.1388769475653255, 1.1156993751209352, 1.093638313080772,
  1.0657171590878205, 1.0362173587708712, 1.0,
  0.9669867858358365, 0.9323750098728378, 0.8958202912590305,
  0.8631993702994263, 0.8253893405524657, 0.7928918905364516,
  0.7666323845128089, 0.7428976357662823, 0.721615762047849
)


In [16]:
tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

IPC,dolar_blue,dolar_oficial,UVA,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.9903031,39.04545,38.43000,2.0014088,201901
1.9174404,38.40250,39.42800,1.9503255,201902
1.8296187,41.63947,42.54210,1.8932303,201903
1.7728863,44.27474,44.35421,1.8247220,201904
1.7212488,46.09546,46.08864,1.7460278,201905
1.6776304,45.06333,44.95500,1.6871348,201906
1.6431248,43.98333,43.75143,1.6361679,201907
1.5814483,54.84286,54.65048,1.5927530,201908
1.4947527,61.05952,58.79000,1.5549163,201909


In [17]:
drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}


In [18]:
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


In [19]:
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}


In [20]:
drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}


In [21]:
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}


In [22]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [23]:
drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}


In [28]:
drift_nati <- function(campos_drift) {
  cat("inicio drift_nati()\n")
  for (campo in campos_drift) {
    cat(campo, " ")
    
    # Calcular el máximo por foto_mes, forzando a tipo double y reemplazando -Inf por NA
    maximos_por_mes <- dataset[, .(max_val = as.numeric(max(get(campo), na.rm = TRUE))), by = foto_mes]
    maximos_por_mes[is.infinite(max_val), max_val := NA_real_]

    # Hacer merge
    dataset <- merge(dataset, maximos_por_mes, by = "foto_mes", all.x = TRUE, sort = FALSE)
    
    # Normalizar y crear variable nueva
    dataset[, paste0(campo, "_nati") := get(campo) / max_val]
    
    # Limpiar
    dataset[, max_val := NULL]
    dataset[, (campo) := NULL]
  }
  cat("fin drift_nati()\n")
}


In [29]:
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like% 
  "^(m|Visa_m|Master_m|vm_m|c)"]

campos_monetarios

[1] "cliente_vip"                         
  [2] "cliente_edad"                        
  [3] "cliente_antiguedad"                  
  [4] "mrentabilidad"                       
  [5] "mrentabilidad_annual"                
  [6] "mcomisiones"                         
  [7] "mactivos_margen"                     
  [8] "mpasivos_margen"                     
  [9] "cproductos"                          
 [10] "ccuenta_corriente"                   
 [11] "mcuenta_corriente_adicional"         
 [12] "mcuenta_corriente"                   
 [13] "ccaja_ahorro"                        
 [14] "mcaja_ahorro"                        
 [15] "mcaja_ahorro_adicional"              
 [16] "mcaja_ahorro_dolares"                
 [17] "cdescubierto_preacordado"            
 [18] "mcuentas_saldo"                      
 [19] "ctarjeta_debito"                     
 [20] "ctarjeta_debito_transacciones"       
 [21] "mautoservicio"                       
 [22] "ctarjeta_visa"                       
 [23] "ctarjeta_visa_transacciones"         
 [24] "mtarjeta_visa_consumo"               
 [25] "ctarjeta_master"                     
 [26] "ctarjeta_master_transacciones"       
 [27] "mtarjeta_master_consumo"             
 [28] "cprestamos_personales"               
 [29] "mprestamos_personales"               
 [30] "cprestamos_prendarios"               
 [31] "mprestamos_prendarios"               
 [32] "cprestamos_hipotecarios"             
 [33] "mprestamos_hipotecarios"             
 [34] "cplazo_fijo"                         
 [35] "mplazo_fijo_dolares"                 
 [36] "mplazo_fijo_pesos"                   
 [37] "cinversion1"                         
 [38] "minversion1_pesos"                   
 [39] "minversion1_dolares"                 
 [40] "cinversion2"                         
 [41] "minversion2"                         
 [42] "cseguro_vida"                        
 [43] "cseguro_auto"                        
 [44] "cseguro_vivienda"                    
 [45] "cseguro_accidentes_personales"       
 [46] "ccaja_seguridad"                     
 [47] "cpayroll_trx"                        
 [48] "mpayroll"                            
 [49] "mpayroll2"                           
 [50] "cpayroll2_trx"                       
 [51] "ccuenta_debitos_automaticos"         
 [52] "mcuenta_debitos_automaticos"         
 [53] "ctarjeta_visa_debitos_automaticos"   
 [54] "mttarjeta_visa_debitos_automaticos"  
 [55] "ctarjeta_master_debitos_automaticos" 
 [56] "mttarjeta_master_debitos_automaticos"
 [57] "cpagodeservicios"                    
 [58] "mpagodeservicios"                    
 [59] "cpagomiscuentas"                     
 [60] "mpagomiscuentas"                     
 [61] "ccajeros_propios_descuentos"         
 [62] "mcajeros_propios_descuentos"         
 [63] "ctarjeta_visa_descuentos"            
 [64] "mtarjeta_visa_descuentos"            
 [65] "ctarjeta_master_descuentos"          
 [66] "mtarjeta_master_descuentos"          
 [67] "ccomisiones_mantenimiento"           
 [68] "mcomisiones_mantenimiento"           
 [69] "ccomisiones_otras"                   
 [70] "mcomisiones_otras"                   
 [71] "cforex"                              
 [72] "cforex_buy"                          
 [73] "mforex_buy"                          
 [74] "cforex_sell"                         
 [75] "mforex_sell"                         
 [76] "ctransferencias_recibidas"           
 [77] "mtransferencias_recibidas"           
 [78] "ctransferencias_emitidas"            
 [79] "mtransferencias_emitidas"            
 [80] "cextraccion_autoservicio"            
 [81] "mextraccion_autoservicio"            
 [82] "ccheques_depositados"                
 [83] "mcheques_depositados"                
 [84] "ccheques_emitidos"                   
 [85] "mcheques_emitidos"                   
 [86] "ccheques_depositados_rechazados"     
 [87] "mcheques_depositados_rechazados"     
 [88] "ccheques_emitidos_rechazados"        
 [89] "mcheques_emitidos_rechazados"      

In [30]:
campos_monetarios <- setdiff(
  campos_monetarios,
  c("cliente_vip", "cliente_edad", "cliente_antiguedad", "clase_ternaria")
)

In [33]:
# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$DR$metodo <- "rank_cero_fijo"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios),
  "drift_nati"   = drift_nati(campos_monetarios)
)


inicio drift_rank_cero_fijo()
mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  cproductos  ccuenta_corriente  mcuenta_corriente_adicional  mcuenta_corriente  ccaja_ahorro  mcaja_ahorro  mcaja_ahorro_adicional  mcaja_ahorro_dolares  cdescubierto_preacordado  mcuentas_saldo  ctarjeta_debito  ctarjeta_debito_transacciones  mautoservicio  ctarjeta_visa  ctarjeta_visa_transacciones  mtarjeta_visa_consumo  ctarjeta_master  ctarjeta_master_transacciones  mtarjeta_master_consumo  cprestamos_personales  mprestamos_personales  cprestamos_prendarios  mprestamos_prendarios  cprestamos_hipotecarios  mprestamos_hipotecarios  cplazo_fijo  mplazo_fijo_dolares  mplazo_fijo_pesos  cinversion1  minversion1_pesos  minversion1_dolares  cinversion2  minversion2  cseguro_vida  cseguro_auto  cseguro_vivienda  cseguro_accidentes_personales  ccaja_seguridad  cpayroll_trx  mpayroll  mpayroll2  cpayroll2_trx  ccuenta_debitos_automaticos  mcuenta_debitos_automaticos  ctarjeta_vis

In [34]:
colnames(dataset)

[1] "numero_de_cliente"                        
  [2] "foto_mes"                                 
  [3] "active_quarter"                           
  [4] "cliente_vip"                              
  [5] "internet"                                 
  [6] "cliente_edad"                             
  [7] "cliente_antiguedad"                       
  [8] "tcuentas"                                 
  [9] "tcallcenter"                              
 [10] "thomebanking"                             
 [11] "tmobile_app"                              
 [12] "Master_delinquency"                       
 [13] "Master_status"                            
 [14] "Master_Fvencimiento"                      
 [15] "Master_Finiciomora"                       
 [16] "Master_fultimo_cierre"                    
 [17] "Master_fechaalta"                         
 [18] "Master_cconsumos"                         
 [19] "Master_cadelantosefectivo"                
 [20] "Visa_delinquency"                         
 [21] "Visa_status"                              
 [22] "Visa_Fvencimiento"                        
 [23] "Visa_Finiciomora"                         
 [24] "Visa_fultimo_cierre"                      
 [25] "Visa_fechaalta"                           
 [26] "Visa_cconsumos"                           
 [27] "Visa_cadelantosefectivo"                  
 [28] "clase_ternaria"                           
 [29] "mrentabilidad_rank"                       
 [30] "mrentabilidad_annual_rank"                
 [31] "mcomisiones_rank"                         
 [32] "mactivos_margen_rank"                     
 [33] "mpasivos_margen_rank"                     
 [34] "cproductos_rank"                          
 [35] "ccuenta_corriente_rank"                   
 [36] "mcuenta_corriente_adicional_rank"         
 [37] "mcuenta_corriente_rank"                   
 [38] "ccaja_ahorro_rank"                        
 [39] "mcaja_ahorro_rank"                        
 [40] "mcaja_ahorro_adicional_rank"              
 [41] "mcaja_ahorro_dolares_rank"                
 [42] "cdescubierto_preacordado_rank"            
 [43] "mcuentas_saldo_rank"                      
 [44] "ctarjeta_debito_rank"                     
 [45] "ctarjeta_debito_transacciones_rank"       
 [46] "mautoservicio_rank"                       
 [47] "ctarjeta_visa_rank"                       
 [48] "ctarjeta_visa_transacciones_rank"         
 [49] "mtarjeta_visa_consumo_rank"               
 [50] "ctarjeta_master_rank"                     
 [51] "ctarjeta_master_transacciones_rank"       
 [52] "mtarjeta_master_consumo_rank"             
 [53] "cprestamos_personales_rank"               
 [54] "mprestamos_personales_rank"               
 [55] "cprestamos_prendarios_rank"               
 [56] "mprestamos_prendarios_rank"               
 [57] "cprestamos_hipotecarios_rank"             
 [58] "mprestamos_hipotecarios_rank"             
 [59] "cplazo_fijo_rank"                         
 [60] "mplazo_fijo_dolares_rank"                 
 [61] "mplazo_fijo_pesos_rank"                   
 [62] "cinversion1_rank"                         
 [63] "minversion1_pesos_rank"                   
 [64] "minversion1_dolares_rank"                 
 [65] "cinversion2_rank"                         
 [66] "minversion2_rank"                         
 [67] "cseguro_vida_rank"                        
 [68] "cseguro_auto_rank"                        
 [69] "cseguro_vivienda_rank"                    
 [70] "cseguro_accidentes_personales_rank"       
 [71] "ccaja_seguridad_rank"                     
 [72] "cpayroll_trx_rank"                        
 [73] "mpayroll_rank"                            
 [74] "mpayroll2_rank"                           
 [75] "cpayroll2_trx_rank"                       
 [76] "ccuenta_debitos_automaticos_rank"         
 [77] "mcuenta_debitos_automaticos_rank"         
 [78] "ctarjeta_visa_debitos_automaticos_rank"   
 [79] "mttarjeta_visa_debitos_automaticos_rank"  
 [80] "ctarjeta_master_debitos_automaticos_rank" 
 [

In [ ]:
# sin codigo en esta primera version del workflow

#### 9.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [35]:
if( !require("lubridate")) install.packages("lubridate", repos = "http://cran.us.r-project.org")
require("lubridate")

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [38]:
install.packages("readxl")
library(readxl)

Installing package into ‘/home/nmarin_datamining/.local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘cellranger’




In [39]:
macro_nati <- read_excel("/content/datasets/macro_nati.xlsx")
head(macro_nati)

foto_mes,ica,base_monetaria,cir_monetaria,ventas_super,ventas_auto,ventas_cc,ventas_ele,ipc,tc_oficial,tc_blue,tc_ccl,brecha,devaluacion,precio_soja
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201901,138.2183,1352805,807109.9,45433.62,6886.632,10001.146,7976.728,189.6101,38.43636,39.04545,37.46727,0.0155995343,0.02013996,249
201902,138.6365,1403081,806108.8,44592.14,7043.166,9122.768,6686.011,196.7501,39.45100,38.40250,38.46250,-0.0273029100,0.02605541,243
201903,138.4144,1351521,804995.2,51065.69,8185.419,11715.035,7821.383,205.9571,42.54895,41.63947,41.36158,-0.0218416230,0.07559572,228
201904,138.6321,1402919,817956.3,49621.41,8112.012,11264.051,6683.572,213.0517,44.36000,44.27474,43.49684,-0.0019257745,0.04168305,216
201905,139.1856,1356452,824343.1,50778.30,8588.680,12324.106,9676.222,219.5691,46.08227,46.09545,45.08409,0.0002859679,0.03809017,220
201906,139.1562,1375803,872460.0,54413.42,8860.500,15925.691,9885.410,225.5370,44.95278,45.06333,43.87833,0.0024533373,-0.02481578,228


In [40]:
# Fusionamos los datasets por la columna 'foto_mes'
dataset <- merge(dataset, macro_nati, by = "foto_mes", all.x = TRUE)

In [44]:
colnames(dataset)

[1] "foto_mes"                                 
  [2] "numero_de_cliente"                        
  [3] "active_quarter"                           
  [4] "cliente_vip"                              
  [5] "internet"                                 
  [6] "cliente_edad"                             
  [7] "cliente_antiguedad"                       
  [8] "tcuentas"                                 
  [9] "tcallcenter"                              
 [10] "thomebanking"                             
 [11] "tmobile_app"                              
 [12] "Master_delinquency"                       
 [13] "Master_status"                            
 [14] "Master_Fvencimiento"                      
 [15] "Master_Finiciomora"                       
 [16] "Master_fultimo_cierre"                    
 [17] "Master_fechaalta"                         
 [18] "Master_cconsumos"                         
 [19] "Master_cadelantosefectivo"                
 [20] "Visa_delinquency"                         
 [21] "Visa_status"                              
 [22] "Visa_Fvencimiento"                        
 [23] "Visa_Finiciomora"                         
 [24] "Visa_fultimo_cierre"                      
 [25] "Visa_fechaalta"                           
 [26] "Visa_cconsumos"                           
 [27] "Visa_cadelantosefectivo"                  
 [28] "clase_ternaria"                           
 [29] "mrentabilidad_rank"                       
 [30] "mrentabilidad_annual_rank"                
 [31] "mcomisiones_rank"                         
 [32] "mactivos_margen_rank"                     
 [33] "mpasivos_margen_rank"                     
 [34] "cproductos_rank"                          
 [35] "ccuenta_corriente_rank"                   
 [36] "mcuenta_corriente_adicional_rank"         
 [37] "mcuenta_corriente_rank"                   
 [38] "ccaja_ahorro_rank"                        
 [39] "mcaja_ahorro_rank"                        
 [40] "mcaja_ahorro_adicional_rank"              
 [41] "mcaja_ahorro_dolares_rank"                
 [42] "cdescubierto_preacordado_rank"            
 [43] "mcuentas_saldo_rank"                      
 [44] "ctarjeta_debito_rank"                     
 [45] "ctarjeta_debito_transacciones_rank"       
 [46] "mautoservicio_rank"                       
 [47] "ctarjeta_visa_rank"                       
 [48] "ctarjeta_visa_transacciones_rank"         
 [49] "mtarjeta_visa_consumo_rank"               
 [50] "ctarjeta_master_rank"                     
 [51] "ctarjeta_master_transacciones_rank"       
 [52] "mtarjeta_master_consumo_rank"             
 [53] "cprestamos_personales_rank"               
 [54] "mprestamos_personales_rank"               
 [55] "cprestamos_prendarios_rank"               
 [56] "mprestamos_prendarios_rank"               
 [57] "cprestamos_hipotecarios_rank"             
 [58] "mprestamos_hipotecarios_rank"             
 [59] "cplazo_fijo_rank"                         
 [60] "mplazo_fijo_dolares_rank"                 
 [61] "mplazo_fijo_pesos_rank"                   
 [62] "cinversion1_rank"                         
 [63] "minversion1_pesos_rank"                   
 [64] "minversion1_dolares_rank"                 
 [65] "cinversion2_rank"                         
 [66] "minversion2_rank"                         
 [67] "cseguro_vida_rank"                        
 [68] "cseguro_auto_rank"                        
 [69] "cseguro_vivienda_rank"                    
 [70] "cseguro_accidentes_personales_rank"       
 [71] "ccaja_seguridad_rank"                     
 [72] "cpayroll_trx_rank"                        
 [73] "mpayroll_rank"                            
 [74] "mpayroll2_rank"                           
 [75] "cpayroll2_trx_rank"                       
 [76] "ccuenta_debitos_automaticos_rank"         
 [77] "mcuenta_debitos_automaticos_rank"         
 [78] "ctarjeta_visa_debitos_automaticos_rank"   
 [79] "mttarjeta_visa_debitos_automaticos_rank"  
 [80] "ctarjeta_master_debitos_automaticos_rank" 
 [

In [41]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

In [45]:
AgregarVariables_IntraMes <- function(dataset) {
  cat("inicio AgregarVariables_IntraMes()\n")
  gc(verbose = FALSE)

  # mes 1 a 12
  if (atributos_presentes(c("foto_mes")))
    dataset[, kmes := foto_mes %% 100]

  # ctrx_quarter normalizado por antigüedad
  if (atributos_presentes(c("ctrx_quarter")))
    dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter)]

  if (atributos_presentes(c("ctrx_quarter", "cliente_antiguedad"))) {
    dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5]
    dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2]
    dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]
  }

  # formato fecha
  if (atributos_presentes(c("foto_mes")))
    dataset[, foto_mes_formato_fecha := as.Date(paste0(substr(foto_mes, 1, 4), "-", substr(foto_mes, 5, 6), "-01"))]

  # payroll sobre edad
  if (atributos_presentes(c("mpayroll_rank", "cliente_edad")))
    dataset[, mpayroll_sobre_edad := mpayroll_rank / cliente_edad]

  # status Mastercard/Visa combinados
  if (atributos_presentes(c("Master_status", "Visa_status"))) {
    dataset[, vm_status01 := pmax(Master_status, Visa_status, na.rm = TRUE)]
    dataset[, vm_status02 := Master_status + Visa_status]
    dataset[, vm_status03 := pmax(ifelse(is.na(Master_status), 10, Master_status),
                                  ifelse(is.na(Visa_status), 10, Visa_status))]
    dataset[, vm_status04 := ifelse(is.na(Master_status), 10, Master_status) +
                             ifelse(is.na(Visa_status), 10, Visa_status)]
    dataset[, vm_status05 := ifelse(is.na(Master_status), 10, Master_status) +
                             100 * ifelse(is.na(Visa_status), 10, Visa_status)]
    dataset[, vm_status06 := ifelse(is.na(Visa_status),
                                    ifelse(is.na(Master_status), 10, Master_status),
                                    Visa_status)]
    dataset[, mv_status07 := ifelse(is.na(Master_status),
                                    ifelse(is.na(Visa_status), 10, Visa_status),
                                    Master_status)]
  }

  # límites combinados
  if (atributos_presentes(c("Master_mfinanciacion_limite_rank", "Visa_mfinanciacion_limite_rank")))
    dataset[, vm_mfinanciacion_limite := rowSums(cbind(Master_mfinanciacion_limite_rank, Visa_mfinanciacion_limite_rank), na.rm = TRUE)]

  # fechas mínimas de vencimiento e inicio mora
  if (atributos_presentes(c("Master_Fvencimiento", "Visa_Fvencimiento")))
    dataset[, vm_Fvencimiento := pmin(Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE)]

  if (atributos_presentes(c("Master_Finiciomora", "Visa_Finiciomora")))
    dataset[, vm_Finiciomora := pmin(Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE)]

  # saldos pesos y dólares
  if (atributos_presentes(c("Master_msaldopesos_rank", "Visa_msaldopesos_rank")))
    dataset[, vm_msaldopesos := rowSums(cbind(Master_msaldopesos_rank, Visa_msaldopesos_rank), na.rm = TRUE)]

  if (atributos_presentes(c("Master_msaldodolares_rank", "Visa_msaldodolares_rank")))
    dataset[, vm_msaldodolares := rowSums(cbind(Master_msaldodolares_rank, Visa_msaldodolares_rank), na.rm = TRUE)]

  # límite de compra
  if (atributos_presentes(c("Master_mlimitecompra_rank", "Visa_mlimitecompra_rank")))
    dataset[, vm_mlimitecompra := rowSums(cbind(Master_mlimitecompra_rank, Visa_mlimitecompra_rank), na.rm = TRUE)]

  # adelantos en pesos
  if (atributos_presentes(c("Master_madelantopesos_rank", "Visa_madelantopesos_rank")))
    dataset[, vm_madelantopesos := rowSums(cbind(Master_madelantopesos_rank, Visa_madelantopesos_rank), na.rm = TRUE)]

  # fecha último cierre
  if (atributos_presentes(c("Master_fultimo_cierre", "Visa_fultimo_cierre")))
    dataset[, vm_fultimo_cierre := pmax(Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE)]

  # consumos combinados
  if (atributos_presentes(c("Master_mconsumospesos_rank", "Visa_mconsumospesos_rank")))
    dataset[, vm_consumos := rowSums(cbind(Master_mconsumospesos_rank, Visa_mconsumospesos_rank), na.rm = TRUE)]

  if (atributos_presentes(c("Master_mconsumosdolares_rank", "Visa_mconsumosdolares_rank")))
    dataset[, vm_consumosd := rowSums(cbind(Master_mconsumosdolares_rank, Visa_mconsumosdolares_rank), na.rm = TRUE)]

  # indicadores derivados
  if (atributos_presentes(c("vm_consumos", "ipc")))
    dataset[, vm_consumos_ipc := vm_consumos / ipc]

  if (atributos_presentes(c("vm_consumos", "ica")))
    dataset[, vm_consumos_ica := vm_consumos / ica]

  if (atributos_presentes(c("vm_consumos", "ventas_cc")))
    dataset[, vm_consumos_ventascc := vm_consumos / ventas_cc]

  if (atributos_presentes(c("vm_consumos", "ventas_super")))
    dataset[, vm_consumos_ventas_super := vm_consumos / ventas_super]

  if (atributos_presentes(c("vm_consumos", "ventas_electro")))
    dataset[, vm_consumos_ventas_electro := vm_consumos / ventas_electro]

  if (atributos_presentes(c("vm_consumos", "cir_monetaria")))
    dataset[, vm_consumos_cirm := vm_consumos / cir_monetaria]

  if (atributos_presentes(c("vm_consumos", "brecha")))
    dataset[, vm_consumos_brecha := vm_consumos / brecha]

  if (atributos_presentes(c("vm_consumos", "devaluacion")))
    dataset[, vm_consumos_devaluacion := vm_consumos / devaluacion]

  if (atributos_presentes(c("vm_consumosd", "tc_oficial")))
    dataset[, vm_consumosd_tc_oficial := vm_consumosd / tc_oficial]

  if (atributos_presentes(c("vm_consumosd", "precio_soja")))
    dataset[, vm_consumosd_soja := vm_consumosd / precio_soja]

  if (atributos_presentes(c("vm_consumosd", "brecha")))
    dataset[, vm_consumosd_brecha := vm_consumosd / brecha]

  # limpieza
  infinitos <- lapply(names(dataset), function(.name) dataset[, sum(is.infinite(get(.name)))])
  if (sum(unlist(infinitos)) > 0) {
    cat("ATENCION, hay valores infinitos en tu dataset. Seran pasados a NA\n")
    dataset[mapply(is.infinite, dataset)] <<- NA
  }

  nans <- lapply(names(dataset), function(.name) dataset[, sum(is.nan(get(.name)))])
  if (sum(unlist(nans)) > 0) {
    cat("ATENCION, hay valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

  cat("fin AgregarVariables_IntraMes()\n")
}


In [46]:
# agrego las variables intra mes

AgregarVariables_IntraMes(dataset)

inicio AgregarVariables_IntraMes()
ATENCION, hay valores infinitos en tu dataset. Seran pasados a NA
ATENCION, hay valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0
fin AgregarVariables_IntraMes()


In [47]:
# visualizo las columas del dataset a esta etapa
ncol(dataset)
colnames(dataset)

[1] 199

[1] "foto_mes"                                 
  [2] "numero_de_cliente"                        
  [3] "active_quarter"                           
  [4] "cliente_vip"                              
  [5] "internet"                                 
  [6] "cliente_edad"                             
  [7] "cliente_antiguedad"                       
  [8] "tcuentas"                                 
  [9] "tcallcenter"                              
 [10] "thomebanking"                             
 [11] "tmobile_app"                              
 [12] "Master_delinquency"                       
 [13] "Master_status"                            
 [14] "Master_Fvencimiento"                      
 [15] "Master_Finiciomora"                       
 [16] "Master_fultimo_cierre"                    
 [17] "Master_fechaalta"                         
 [18] "Master_cconsumos"                         
 [19] "Master_cadelantosefectivo"                
 [20] "Visa_delinquency"                         
 [21] "Visa_status"                              
 [22] "Visa_Fvencimiento"                        
 [23] "Visa_Finiciomora"                         
 [24] "Visa_fultimo_cierre"                      
 [25] "Visa_fechaalta"                           
 [26] "Visa_cconsumos"                           
 [27] "Visa_cadelantosefectivo"                  
 [28] "clase_ternaria"                           
 [29] "mrentabilidad_rank"                       
 [30] "mrentabilidad_annual_rank"                
 [31] "mcomisiones_rank"                         
 [32] "mactivos_margen_rank"                     
 [33] "mpasivos_margen_rank"                     
 [34] "cproductos_rank"                          
 [35] "ccuenta_corriente_rank"                   
 [36] "mcuenta_corriente_adicional_rank"         
 [37] "mcuenta_corriente_rank"                   
 [38] "ccaja_ahorro_rank"                        
 [39] "mcaja_ahorro_rank"                        
 [40] "mcaja_ahorro_adicional_rank"              
 [41] "mcaja_ahorro_dolares_rank"                
 [42] "cdescubierto_preacordado_rank"            
 [43] "mcuentas_saldo_rank"                      
 [44] "ctarjeta_debito_rank"                     
 [45] "ctarjeta_debito_transacciones_rank"       
 [46] "mautoservicio_rank"                       
 [47] "ctarjeta_visa_rank"                       
 [48] "ctarjeta_visa_transacciones_rank"         
 [49] "mtarjeta_visa_consumo_rank"               
 [50] "ctarjeta_master_rank"                     
 [51] "ctarjeta_master_transacciones_rank"       
 [52] "mtarjeta_master_consumo_rank"             
 [53] "cprestamos_personales_rank"               
 [54] "mprestamos_personales_rank"               
 [55] "cprestamos_prendarios_rank"               
 [56] "mprestamos_prendarios_rank"               
 [57] "cprestamos_hipotecarios_rank"             
 [58] "mprestamos_hipotecarios_rank"             
 [59] "cplazo_fijo_rank"                         
 [60] "mplazo_fijo_dolares_rank"                 
 [61] "mplazo_fijo_pesos_rank"                   
 [62] "cinversion1_rank"                         
 [63] "minversion1_pesos_rank"                   
 [64] "minversion1_dolares_rank"                 
 [65] "cinversion2_rank"                         
 [66] "minversion2_rank"                         
 [67] "cseguro_vida_rank"                        
 [68] "cseguro_auto_rank"                        
 [69] "cseguro_vivienda_rank"                    
 [70] "cseguro_accidentes_personales_rank"       
 [71] "ccaja_seguridad_rank"                     
 [72] "cpayroll_trx_rank"                        
 [73] "mpayroll_rank"                            
 [74] "mpayroll2_rank"                           
 [75] "cpayroll2_trx_rank"                       
 [76] "ccuenta_debitos_automaticos_rank"         
 [77] "mcuenta_debitos_automaticos_rank"         
 [78] "ctarjeta_visa_debitos_automaticos_rank"   
 [79] "mttarjeta_visa_debitos_automaticos_rank"  
 [80] "ctarjeta_master_debitos_automaticos_rank" 
 [

#### 9.3.1.4  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [48]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [49]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")


In [50]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}


In [ ]:
# Feature Engineering Historico

setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria","ica", "base_monetaria", "cir_monetaria", "ventas_super", "ventas_auto","ventas_cc","ventas_ele", "ipc", "tc_oficial", "tc_blue", "tc_ccl", "brecha", "devaluacion","precio_soja"
)
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE


In [ ]:
cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}



Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

#### 9.3.1.5  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest



In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [ ]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$FE_rf$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


In [ ]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [ ]:
# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

In [ ]:
ncol(dataset)
colnames(dataset)

#### 9.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  ni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

El objetivo de esta etapa NO es mejorar el modelo predictivo

El objetivo es eliminar campos poco importantes para hacer espacio a nuevos campos, debido a las restricciones de memoria RAM.

In [ ]:
VPOS_CORTE <- c()

fganancia_lgbm_meseta <- function(probs, datos) {
  vlabels <- get_field(datos, "label")
  vpesos <- get_field(datos, "weight")

  tbl <- as.data.table(list(
    "prob" = probs,
    "gan" = ifelse(vlabels == 1 & vpesos > 1, PARAM$CN$train$gan1, PARAM$CN$train$gan0)
  ))

  setorder(tbl, -prob)
  tbl[, posicion := .I]
  tbl[, gan_acum := cumsum(gan)]
  setorder(tbl, -gan_acum) # voy por la meseta

  gan <- mean(tbl[1:500, gan_acum]) # meseta de tamaño 500

  pos_meseta <- tbl[1:500, median(posicion)]
  VPOS_CORTE <<- c(VPOS_CORTE, pos_meseta)

  return(list(
    "name" = "ganancia",
    "value" = gan,
    "higher_better" = TRUE
  ))
}


In [ ]:
# Elimina del dataset las variables que estan por debajo
#  de la capa geologica de canaritos
# se llama varias veces, luego de agregar muchas variables nuevas,
#  para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1

campitos <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

CanaritosAsesinos <- function(
  canaritos_ratio,
  canaritos_desvios,
  canaritos_semilla) {

  cat( "inicio CanaritosAsesinos()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$CN$train$clase01_valor1,
      clase01 := 1L ]

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  for (i in 1:(ncol(dataset) * canaritos_ratio)) {
    dataset[, paste0("canarito", i) := runif(nrow(dataset))]
  }

  campos_buenos <- setdiff(
    colnames(dataset),
    c( campitos, "clase01")
  )

  azar <- runif(nrow(dataset))

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$CN$train$training &
      (clase01 == 1 | azar < PARAM$CN$train$undersampling))]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    weight = dataset[
      entrenamiento == TRUE,
      ifelse(clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )

  dvalid <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$CN$train$validation, campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$CN$train$validation, clase01],
    weight = dataset[
      foto_mes %in% PARAM$CN$train$validation,
      ifelse( clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )


  param <- list(
    objective = "binary",
    metric = "custom",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    seed = canaritos_semilla,
    max_depth = -1, # -1 significa no limitar,  por ahora lo dejo fijo
    min_gain_to_split = 0.0, # por ahora, lo dejo fijo
    lambda_l1 = 0.0, # por ahora, lo dejo fijo
    lambda_l2 = 0.0, # por ahora, lo dejo fijo
    max_bin = 31, # por ahora, lo dejo fijo
    num_iterations = 9999, # un numero grande, lo limita early_stopping_rounds
    force_row_wise = TRUE, # para que los alumnos no se atemoricen con  warning
    learning_rate = 0.065,
    feature_fraction = 1.0, # lo seteo en 1
    min_data_in_leaf = 260,
    num_leaves = 60,
    early_stopping_rounds = 200,
    num_threads = 1
  )

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  modelo <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalid),
    eval = fganancia_lgbm_meseta,
    param = param,
    verbose = -100
  )

  tb_importancia <- lgb.importance(model = modelo)
  tb_importancia[, pos := .I]

  fwrite(tb_importancia,
    file = paste0("impo_", GVEZ, ".txt"),
    sep = "\t"
  )

  GVEZ <<- GVEZ + 1

  umbral <- tb_importancia[
    Feature %like% "canarito",
    median(pos) + canaritos_desvios * sd(pos)
  ] # Atencion corto en la mediana mas desvios!!

  col_utiles <- tb_importancia[
    pos < umbral & !(Feature %like% "canarito"),
    Feature
  ]

  col_utiles <- unique(c(
    col_utiles,
    c(campitos, "mes")
  ))

  col_inutiles <- setdiff(colnames(dataset), col_utiles)

  dataset[, (col_inutiles) := NULL]

  cat( "fin CanaritosAsesinos()\n")

  return( tb_importancia )
}


In [ ]:
# Estos DOS parametros son los que se deben modificar
PARAM$CN$ratio <- 0.5
PARAM$CN$desvios <- 3


# Parametros quasi fijos
# Parametros de un LightGBM que se genera para estimar la column importance
PARAM$CN$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$CN$train$positivos <- c( "BAJA+2")
PARAM$CN$train$training <- c( 202101, 202102, 202103)
PARAM$CN$train$validation <- c( 202105 )
PARAM$CN$train$undersampling <- 0.1
PARAM$CN$train$gan1 <- 117000
PARAM$CN$train$gan0 <-  -3000

In [ ]:
# la llamada a Canaritos Asesinos
tb_importancia <- CanaritosAsesinos(
  canaritos_ratio = PARAM$CN$ratio,
  canaritos_desvios = PARAM$CN$desvios,
  canaritos_semilla = PARAM$semilla_primigenia
)


In [ ]:
# grabo la importancia, ver el archivo directamente en la carpeta

fwrite( tb_importancia,
  file="canaritos.txt",
  sep="\t"
)

In [ ]:
# verifico
ncol(dataset)
colnames(dataset)

### 9.3.2 Modelado

#### 9.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
PARAM$trainingstrategy$validate <- c(202106,202107)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104
)

PARAM$trainingstrategy$training_pct <- 0.1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

In [ ]:
colnames(dtrain)

####  9.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 70

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)



Nuevos  pseudo hiperparámetros

*   leaf_size (0.0,  0.5]  es el   min_data_in_leaf / nrow(dataset)
*   coverage (0.0, 1.0]  se cumple   num_leaves =   coverage * (nrow(dataset)/min_data_in_leaf)



In [ ]:
# Notar que se usan los pseudo hiperparametros "ratio"  leaf_size  y coverage
#  que luego terminan en  min_data_in_leaf y  num_leaves
#  de forma que sea invariante al tamaño del dataset

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.02, upper = 0.3),
  makeNumericParam("feature_fraction", lower = 0.05, upper = 0.90),
  makeNumericParam("coverage", lower = 0.05, upper = 1.0), # nuevo
  makeNumericParam("leaf_size", lower = 0.001, upper = 0.2) # nuevo
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # hago la transformacion de leaf_size y  coverage
  if( "leaf_size"  %in% names(param_completo) &
      "coverage"  %in% names(param_completo)
  )
  {
    # primero defino el tamaño de las hojas
    param_completo$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_completo$leaf_size )  )
    # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
    param_completo$num_leaves <- pmin( 131072,
      pmax( 8,  round( ( param_completo$coverage * nrow( dtrain ) / param_completo$min_data_in_leaf ) ) ))
  }

  if( "leaf_size"  %in% names(param_completo) &
      !( "coverage"  %in% names(param_completo) )
  )
  {
    # primero defino el tamaño de las hojas
    param_completo$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_completo$leaf_size )  )
  }


  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(learning_rate, feature_fraction, coverage, leaf_size)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

### 9.3.3 Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

#### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

# hago la transformacion de leaf_size y  coverage
  if( "leaf_size"  %in% names(param_final) &
      "coverage"  %in% names(param_final)
  )
  {
    # primero defino el tamaño de las hojas
    param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dfinal_train) * param_final$leaf_size )  )
    # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
    param_final$num_leaves <- pmin( 131072,
      pmax( 8,  round( ( param_final$coverage * nrow( dfinal_train ) / param_final$min_data_in_leaf ) ) ))
  }

  if( "leaf_size"  %in% names(param_final) &
      !( "coverage"  %in% names(param_final) )
  )
  {
    # primero defino el tamaño de las hojas
    param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dfinal_train) * param_final$leaf_size )  )
  }

In [ ]:
param_final

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
PARAM$FT$semillerio <- 60  # cantidad de semillas

In [ ]:
if(!require("primes")) install.packages("primes")
require("primes")

In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$FT$semillerio)]

cat( PARAM$FT$semillas)

In [ ]:
dir.create("modelos", showWarnings =FALSE)

primero <- TRUE
for( sem in PARAM$FT$semillas)
{
  nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
  if( !file.exists(nombre_arch) )
  {
    param_final$seed <- sem

    set.seed(sem, kind = "L'Ecuyer-CMRG")
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    lgb.save(final_model, nombre_arch) # grabo el modelo"

    # grabo la primer importancia de variables
    #  Natalia : da lo mismo cual se guarda
    if( primero)
    {
      primero <- FALSE
      tb_importancia <- as.data.table(lgb.importance(final_model))
      archivo_importancia <- "impo.txt"

      fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
      )
    }
 }
}

### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

In [ ]:
# veo que hay en tb_prediccion
tb_prediccion

In [ ]:
# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "labo-i-2025-rosario-analista-jr"   o  la original "labo-i-2025-rosario"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-rosario"
PARAM$kaggle$cortes <- 10*seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0(
    "./kaggle/KA",
    PARAM$experimento, "_",
    "s", length(PARAM$FT$semillas), "_",
    envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")